In [1]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import math

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome()

In [3]:
stealth(driver,
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
)

In [4]:
driver.get("https://animepahe.ru")

# input = driver.find_element(By.CLASS_NAME, "input-search")
# input.send_keys("One piece")

In [5]:
# Get search results
# search_result_wrap = driver.find_element(By.CLASS_NAME, "search-results-wrap")

# while True:
#     try:
#         ul =  search_result_wrap.find_element(By.CLASS_NAME, "search-results")
#         li = ul.find_elements(By.TAG_NAME, "li")
#         for anime in li:
#             a = anime.find_element(By.TAG_NAME, "a")
#             result_title = a.find_element(By.CLASS_NAME, "result-title")
#             print (result_title.text.strip())
#         break
#     except Exception as e:
#         print (f"Couldn't find the results. Retrying in 5s: {e}")
#         time.sleep(5)

In [6]:
import requests

response = requests.get("https://animepahe.ru/api?m=search&q=One piece")
jsonres = None

if response.status_code == 200:
    # Print the response content (usually in JSON format for APIs)
    jsonres = response.json()
    print(jsonres)
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code} - {response.text}")

{'total': 61, 'per_page': 8, 'current_page': 1, 'last_page': 8, 'from': 1, 'to': 8, 'data': [{'id': 4, 'title': 'One Piece', 'type': 'TV', 'episodes': 0, 'status': 'Currently Airing', 'season': 'Fall', 'year': 1999, 'score': 8.71, 'poster': 'https://i.animepahe.ru/posters/355e6e3127aa31f0d806114169b52c4fb6da4b87df7f9c1809b9e3de97b8aac5.jpg', 'session': '1f0c2e22-a432-cda9-7ac8-e6cbc7852a93'}, {'id': 3778, 'title': 'One Piece Movie 01', 'type': 'Movie', 'episodes': 1, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2000, 'score': 7.09, 'poster': 'https://i.animepahe.ru/posters/239bba86be7f81f2f7bf011f997136f67cecfa0fcaaac4b0747cdadefb45e845.jpg', 'session': 'dcdc6e99-efcc-8a0a-505f-5d93ce7c0ace'}, {'id': 3838, 'title': 'One Piece Film: Gold - Cine Mike Popcorn Kokuchi', 'type': '', 'episodes': 1, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2016, 'score': 6.36, 'poster': 'https://i.animepahe.ru/posters/7eaa1bebf5c244a8a34e95c0d3eb45d3cc0801f55c8f0e194e8b9934e8e228a0

In [7]:
driver.get(f"https://animepahe.ru/anime/{jsonres['data'][0]['session']}")

In [8]:
response = requests.get(f"https://animepahe.ru/api?m=release&id={jsonres['data'][0]['session']}&sort=episode_asc&page=1")
anime_details = response.json()

anime_details

{'total': 1089,
 'per_page': 30,
 'current_page': 1,
 'last_page': 37,
 'next_page_url': 'https://animepahe.ru/api?page=2',
 'prev_page_url': None,
 'from': 1,
 'to': 30,
 'data': [{'id': 36600,
   'anime_id': 4,
   'episode': 1,
   'episode2': 0,
   'edition': '',
   'title': '',
   'snapshot': 'https://i.animepahe.ru/snapshots/c311c5b993085c3e9de8e44387a0cb48b69fe2c3ba4fcf77d52f1ab4f890c12a.jpg',
   'disc': '',
   'audio': 'jpn',
   'duration': '00:25:00',
   'session': '7dcd6ed708bd2cf222a21ec3be167142043692ee3c05a6bdd4a89a4cfeaebdf5',
   'filler': 0,
   'created_at': '2019-12-12 09:53:44'},
  {'id': 36601,
   'anime_id': 4,
   'episode': 2,
   'episode2': 0,
   'edition': '',
   'title': '',
   'snapshot': 'https://i.animepahe.ru/snapshots/ba684b711d8d1382d9934b69fac36dd8a55bcd1d81cd04ea668f1173fc503761.jpg',
   'disc': '',
   'audio': 'jpn',
   'duration': '00:25:00',
   'session': '538a18d996aa035b044cfd23ce6517d0ef226f3d1ef8eea2678a9a32c62bb6ff',
   'filler': 0,
   'created_at':

In [9]:
# getting the episodes
def get_episode_link(episode, total):
    page = math.ceil(episode/30)
    response = requests.get(f"https://animepahe.ru/api?m=release&id={jsonres['data'][0]['session']}&sort=episode_asc&page={page}").json()
    position = (episode - (page - 1) * 30) - 1
    return f"https://animepahe.ru/play/{jsonres['data'][0]['session']}/{response['data'][position]['session']}"
        
link = get_episode_link(60, anime_details['total'])
link

'https://animepahe.ru/play/1f0c2e22-a432-cda9-7ac8-e6cbc7852a93/a1f3c490883a2f81cd39f1c915b8f0487d1827687d15531263ac3f600f331a77'

In [10]:

requests.get(
    "https://animepahe.ru/play/1f0c2e22-a432-cda9-7ac8-e6cbc7852a93/a1f3c490883a2f81cd39f1c915b8f0487d1827687d15531263ac3f600f331a77",
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15'
    }
).text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<link rel="alternate" href="https://animepahe.ru" hreflang="en-us">\n<meta name="viewport" content="width=device-width,initial-scale=1">\n<meta name="theme-color" content="#373a3c">\n<meta name="msapplication-navbutton-color" content="#373a3c">\n<meta name="apple-mobile-web-app-status-bar-style" content="#373a3c">\n<meta http-equiv="x-dns-prefetch-control" content="on">\n<link rel="preconnect" href="//i.animepahe.ru">\n<link rel="preload" href="/app/fonts/QldONTRRphEb_-V7LB6xTA.woff2" as="font" type="font/woff2" crossorigin>\n<link rel="preload" href="/app/css/bootstrap.min.css" as="style">\n<link rel="preload" href="/app/css/fork-awesome.min.css" as="style">\n<link rel="preload" href="/app/css/style.css" as="style">\n<link rel="preload" href="/app/css/yuki.css" as="style">\n<link rel="preload" href="/app/js/vendor/bootstrap.bundle.min.js" as="script">\n<link rel="preload" href="/app/js/cor

In [11]:
driver.get(link)

In [12]:
# Get quality lists
qualities = []

for a in driver.find_elements(By.CSS_SELECTOR, "#pickDownload .dropdown-item"):
    quality = {
        "name": a.get_attribute("innerHTML").split(" ")[2],
        "link": a.get_attribute("href")
    }
    qualities.append(quality)

qualities

[{'name': '720p', 'link': 'https://pahe.win/zljrN'},
 {'name': '1080p', 'link': 'https://pahe.win/BQzVb'}]

In [13]:
qualities[0]['link']

'https://pahe.win/zljrN'

In [14]:
driver.get(qualities[0]['link'])

In [15]:
# Get the continue button
link_to_download = ""

while True:
    try:
        a = driver.find_element(By.XPATH, "//a[text()='Continue']")
        link_to_download = a.get_attribute("href")
        break
    except:
        print("Couldn't find button, waiting 7s")
        time.sleep(7)
 
link_to_download

Couldn't find button, waiting 7s


'https://kwik.cx/f/Og4bTrEAaWNX'

In [16]:
driver.get(link_to_download)

In [17]:
# Find button and click it
link = driver.find_element(By.CSS_SELECTOR, ".main .download form").get_attribute("action")
token = driver.find_element(By.CSS_SELECTOR, ".main .download form input").get_attribute("value")
print("Link:", link)
print("Token:", token)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.129)
Stacktrace:
0   chromedriver                        0x0000000102f184dc chromedriver + 4162780
1   chromedriver                        0x0000000102f10664 chromedriver + 4130404
2   chromedriver                        0x0000000102b67bc0 chromedriver + 293824
3   chromedriver                        0x0000000102b406c0 chromedriver + 132800
4   chromedriver                        0x0000000102bd925c chromedriver + 758364
5   chromedriver                        0x0000000102bed294 chromedriver + 840340
6   chromedriver                        0x0000000102ba16bc chromedriver + 530108
7   chromedriver                        0x0000000102ba2930 chromedriver + 534832
8   chromedriver                        0x0000000102edde08 chromedriver + 3923464
9   chromedriver                        0x0000000102ee23dc chromedriver + 3941340
10  chromedriver  

In [ ]:
# cookies = {}
# for cookie in driver.get_cookies():
#     cookies[cookie['name']] = cookie['value']
# cookies

cookies = '; '.join([
    f"{cookie['name']}={cookie['value']}"
    for cookie in driver.get_cookies()
])
cookies

In [ ]:
data = {
    "_token": token
}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0',
  'Referer': 'https://kwik.cx/f/r6gCLSqIe9Nh',
  'Origin': 'https://kwik.cx',
  'Host': 'kwik.cx',
  'Cookie': cookies
}

In [ ]:

data = {
    "_token": token
}
response = requests.post(link, data=data, headers=headers, stream=True)
file_name = response.headers.get('Content-Disposition').split('=')[1]
print(file_name)

response.raise_for_status()

total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kibibyte

progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

with open(file_name, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)

progress_bar.close()